# Creating the weather file 

This script displays the method created to develop the weather file used as input in bifacial_radiance, both with 5-minute intervals and hourly intervals. Measured data for GHI, DHI, DNI, and albedo were extracted and used to replace clear-sky data for the defined timestamps.

The script shows the weather file created is for 2023, where measured data between the 18th of January and the 31st of March is used. For the periods with measured data in 2022, the same script was used, but with different start- and end-times for the clear-sky dataframe and the measured data.

This script has also been used on measured GHI, DHI, DNI, and albedo data that have been through the uncertainty analysis to create weather files for the upper and lower limit of irradiance when the measurement uncertainty has been taken into account.

### Importing necessary packages

In [ ]:
import numpy as np
import pandas as pd
import pvlib

### Defining the parameters

In [ ]:
starttime = '2023-01-01'   # YYYY-mm-dd
endtime = '2023-12-31'     # YYYY-mm-dd
freq = '5T'                # 5-minute interval between the timestamps

lat = 59.97315             # Latitude of the test site  
lon = 11.05166             # Longitude of the test site 
tz = 0                     # Wanted timezone, UTC 
alt = 214                  # Altitude 
loc = 'Kjeller'            # Location of the test site

### Calculating clear-sky values for GHI, DHI, and DNI

In [ ]:
times = pd.date_range(start, end, freq)                   # Creating the index

site = pvlib.location.Location(lat, lon, tz, alt, loc)    # Defining the location
clearsky = site.get_clearsky(times)                       # Calculating clear-sky values for GHI, DHI, and DNI
clearsky = clearsky.tz_localize('UTC')                    # Setting the timesone to UTC

### Creating the datafreame and filling in clear-sky values

In [ ]:
clearsky_data = pd.DataFrame()
clearsky_data['GHI'] = clearsky['ghi']
clearsky_data['DHI'] = clearsky['dhi']
clearsky_data['DNI'] = clearsky['dni']
clearsky_data['Albedo' ] = 0.15

### Retrieve the measured data with a 5-minute interval 

In [ ]:
data = pd.read_csv(r'path...\filename.csv', sep=';')                      # Importing the file with measured data
data = data.set_index('Timestamp')                                        # Setting the timestamps to be the index
data.index = pd.to_datetime(data.index)                                   # Converting the index to date-time
data = data.tz_localize('CET', ambiguous = False, nonexistent = 'NaT')    # Localize timezone and handling daylight saving time
data.loc[data.index.dropna()]                                             # Removing NaN values
data = data.tz_convert('UTC')                                             # Converting the timezone to UTC

#Renaming the needed columns to match the clearsky dataframe
data.rename(columns = {'Roof_albedo_downwell [W/m²]':'Irradiance_upwell [W/m²]',
                       'Roof_albedo_upwell [W/m²]':'Irradiance_downwell [W/m²]',
                       'Solys_ghi [W/m²] - SRAD':'GHI',
                       'Solys_dhi [W/m²] - SRAD':'DHI',
                       'Solys_dni [W/m²] - SRAD':'DNI'}, inplace=True)

data['Albedo'] = abs(data['Irradiance_upwell [W/m²]'] / data['Irradiance_downwell [W/m²]'])   # Calculating the albedo
data['Albedo'] = data['Albedo'].replace(np.nan, 0)                                            # Setting NaN values to be 0

### Replacing the clear-sky data with measured data for the timperiode were measured data have been retrieved

In [ ]:
start = '2023-01-17 23:00:00+00:00'    #Start-time for measured values with the format YYYY-MM-DD HH:mm:ss+timezone
end = '2023-03-31 21:55:00+00:00'      #End-time for measured values with the format YYYY-MM-DD HH:mm:ss+timezone

#Making a copy of the clearsky dataframe
raw_tmy = clearsky_data.copy()      

#Replacing clear-sky data with measured data for the given period of time
for column in raw_tmy.columns:
    if column in data.columns:
        raw_tmy[column].loc[start:end] = data[column].loc[start:end]
              
#If the measued irradiance is <=0 it is set to be 0
raw_tmy['DHI'].loc[raw_tmy['DHI'] <= 0] = 0
raw_tmy['GHI'].loc[raw_tmy['GHI'] <= 0] = 0
raw_tmy['DNI'].loc[raw_tmy['DNI'] <= 0] = 0

### Creating a file with the columns of a TMY-file on a 5-minute interval

In [ ]:
#Defining the column names
tmy_columns = ['Date (MM/DD/YYYY)', 'Time (HH:MM)', 'GHI', 'DNI', 'DHI', 'Alb', 'DryBulb', 'Wspd', 'Wdir']

#Filling in data in the file
tmy_df = pd.DataFrame(index=raw_tmy.index, columns=tmy_columns, data=np.nan)    
tmy_df['GHI'] = raw_tmy['GHI']  
tmy_df['DNI'] = raw_tmy['DNI']
tmy_df['DHI'] = raw_tmy['DHI']    
tmy_df['Alb'] = raw_tmy['Albedo']    
tmy_df['DryBulb'] = 0
tmy_df['Wspd'] = 0  
tmy_df['Wdir'] = 0   
tmy_df['Date (MM/DD/YYYY)'] = raw_tmy.index.strftime('%m/%d/%Y')  
tmy_df['Time (HH:MM)'] = raw_tmy.index.strftime('%H:%M')

#Saving the file to wanted path and with the wanted filename
tmy_df.to_csv(r'path...\filename.csv', header = True, index = False)    

### Creating a similar file on a hourly interval using instantaneous values

In [ ]:
raw_tmy_hour = raw_tmy.loc[raw_tmy.index.strftime('%M:%S') == '30:00']   #Selecting hourly data for 11:30 PM, 12:30 AM etc. 
raw_tmy_hour.index = (raw_tmy_hour.index + pd.Timedelta(minutes=30))     #Converting the index to get right-handled data

#Filling in data in the file
tmy_df_hour = pd.DataFrame(index=raw_tmy_hour.index, columns=tmy_columns, data=np.nan)    
tmy_df_hour['GHI'] = raw_tmy_hour['GHI']  
tmy_df_hour['DNI'] = raw_tmy_hour['DNI']
tmy_df_hour['DHI'] = raw_tmy_hour['DHI']    
tmy_df_hour['Alb'] = raw_tmy_hour['Albedo']    
tmy_df_hour['Date (MM/DD/YYYY)'] = raw_tmy_hour.index.strftime('%m/%d/%Y')  
tmy_df_hour['Time (HH:MM)'] = raw_tmy_hour.index.strftime('%H:%M')
tmy_df_hour['DryBulb'] = 0
tmy_df_hour['Wspd'] = 0  
tmy_df_hour['Wdir'] = 0   

#Saving the file to wanted path and with the wanted filename
tmy_df.to_csv(r'path...\filename.csv', header = True, index = False)    

### Printing the header which need to be added manually to the files

In [ ]:
print('Remember to manually add header: 0, Name, Country, {:}, {:}, {:}, {:}'.format(tz, latitude, longitude, altitude))